<a href="https://colab.research.google.com/github/StevenLomon/face-recognition-project/blob/CNN-model-development/face_recognition_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import imghdr
from imutils import paths
from matplotlib import pyplot as plt
#from tensorflow.keras.datasets import cifar10

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainingpath = '/Users/yari/2023/Applicerad_AI/fairfacedata/structured_images/train'
testpath = '/Users/yari/2023/Applicerad_AI/fairfacedata/structured_images/test'

# image preprocessing

In [ ]:
def rotate_image(image, angle):
    """Rotate the image by a certain angle."""
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

In [ ]:

def crop_image(image, start_x=0, start_y=0, target_size=(64, 64)):
    """Crop the image to a rectangle and resize to the target size."""
    cropped_image = image[start_y:start_y+target_size[1], start_x:start_x+target_size[0]]
    # Resize the image to the target size if it's not already that size
    if cropped_image.shape[0] != target_size[1] or cropped_image.shape[1] != target_size[0]:
        cropped_image = cv2.resize(cropped_image, target_size, interpolation=cv2.INTER_AREA)
    return cropped_image

In [ ]:
def blur_image(image, k_size=5):
    """Blur the image using Gaussian Blur."""
    blurred_image = cv2.GaussianBlur(image, (k_size, k_size), 0)
    return blurred_image

In [ ]:
def change_color(image, flag=cv2.COLOR_BGR2GRAY):
    """Change the color of the image."""
    # Converting the image to grayscale or another color space
    converted_image = cv2.cvtColor(image, flag)
    return converted_image

In [ ]:
def custom_augmentation(image):
    image = rotate_image(image, angle=np.random.randint(-15, 15))
    image = crop_image(image)
    if np.random.rand() < 0.3:
        image = blur_image(image, k_size=5)
    return image

## splitting the data into train and test sets

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

target_size = (64, 64)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        preprocessing_function =custom_augmentation
    )

test_datagen = ImageDataGenerator(rescale=1./255)



training_set = train_datagen.flow_from_directory(
        trainingpath,
        target_size=target_size,
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        testpath,
        target_size=target_size,
        batch_size=32,
        class_mode='categorical')

## plotting a sample batch


In [ ]:
import matplotlib.pyplot as plt
temp_generator = train_datagen.flow_from_directory(
    trainingpath,
    target_size=target_size,
    batch_size=1,
    class_mode='categorical'
)

In [ ]:
for i in range(10):
    img, label = temp_generator.next()
    img = img[0]
    plt.imshow(img)
    plt.title(f"Sample {i+1}")
    plt.show()

In [ ]:
test_set.class_indices

In [ ]:
TrainClasses=training_set.class_indices

In [ ]:
print(TrainClasses)

In [ ]:
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

In [ ]:
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

In [ ]:
print("Mapping of Face and its ID",ResultMap)

In [ ]:
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

# Creating CNN model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
input_shape = (64, 64, 3)
OutputNeurons = 7

classifier = Sequential()


classifier.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(256, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(OutputNeurons, activation='softmax'))

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
classifier.summary()

In [ ]:
import time
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, mode='min', verbose=1)

StartTime = time.time()
classifier.fit(
                    training_set,
                    steps_per_epoch=885,
                    epochs=1000,
                    validation_data=test_set,
                    validation_steps=221,
                    callbacks=[early_stopping]
                    )

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')